In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from sklearn.metrics import mean_absolute_error
from jarvis.db.figshare import data


import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.ticker as ticker
from matplotlib.font_manager import FontProperties
%matplotlib inline

### Property Value Stats

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
# MAD
# props = ['formation_energy_peratom','ehull','magmom_outcar', 'mbj_bandgap','spillage', 'slme','Tc_supercon']

# props = ['formation_energy_peratom']

# for prop in props:

#     t_file_path = f"./data/dataset_alignn_bert-base-uncased_chemnlp_prop_{prop}_test.csv"
#     df_t = pd.read_csv(t_file_path, index_col = 0)

#     tr_file_path = f"./data/dataset_alignn_bert-base-uncased_chemnlp_prop_{prop}_train.csv"
#     df_tr = pd.read_csv(tr_file_path, index_col = 0)
    
#     v_file_path = f"./data/dataset_alignn_bert-base-uncased_chemnlp_prop_{prop}_val.csv"
#     df_v = pd.read_csv(v_file_path, index_col = 0)
#     df_train = pd.concat([df_tr, df_v],axis = 0)
   
    # tr_mean = np.mean(df_train[prop].values)
    # n = len(df_t[prop].values)
    # mad = mean_absolute_error(df_t[prop].values, n * [tr_mean])
    # print(f"{prop} MAD: {mad}")
    # print(f"dataset size {prop}: {df_tr.shape[0] + df_t.shape[0] + df_v.shape[0]}")
    






# List of properties
props = ['formation_energy_peratom','ehull','magmom_outcar', 'mbj_bandgap','spillage', 'slme','Tc_supercon']
units = {'formation_energy_peratom': 'eV/atom', 'ehull': 'eV/atom', 'magmom_outcar': 'mu_B', 'mbj_bandgap': 'eV', 'spillage': '', 'slme': '%', 'Tc_supercon': 'K'}
# props = ['mbj_bandgap']
# Initialize the figure
fig, axes = plt.subplots(nrows=2, ncols=4, figsize=(20, 10))
axes = axes.flatten()

for i, prop in enumerate(props):
    # File paths
    t_file_path = f"./data/dataset_alignn_bert-base-uncased_chemnlp_prop_{prop}_test.csv"
    tr_file_path = f"./data/dataset_alignn_bert-base-uncased_chemnlp_prop_{prop}_train.csv"
    v_file_path = f"./data/dataset_alignn_bert-base-uncased_chemnlp_prop_{prop}_val.csv"

    # Read the data
    df_t = pd.read_csv(t_file_path, index_col=0)
    df_tr = pd.read_csv(tr_file_path, index_col=0)
    df_v = pd.read_csv(v_file_path, index_col=0)
    
    
    # Create a DataFrame with an additional column for the dataset type
    df_tr['dataset'] = 'train'
    df_v['dataset'] = 'val'
    df_t['dataset'] = 'test'
    
    # Concatenate all data
    df_all = pd.concat([df_tr, df_v, df_t], axis=0)
    
    # Plot the violin plot with box plot inside
    sns.violinplot(x='dataset', y=prop, data=df_all, ax=axes[i], inner="point", palette='muted')
    # sns.boxplot(x='dataset', y=prop, data=df_all, ax=axes[i], width=0.1, palette='dark')
    # sns.stripplot(x='dataset', y=prop, data=df_all, ax=axes[i], jitter=True, size=2, color='k', alpha=0.5)

    # Set title
    axes[i].set_title(prop)
    axes[i].set_ylabel(units[prop])
    axes[i].grid(True, which='both', linestyle='--', linewidth=0.5)

# Adjust layout
plt.tight_layout()
plt.savefig('all_violin.png', dpi=600)
plt.show()


In [ ]:
# Create a figure with 4x4 subplots
props = ['ehull','mbj_bandgap', 'slme', 'spillage', 'magmom_outcar','formation_energy_peratom', 'Tc_supercon']
rows = ["gpt2", "bert-base-uncased", "facebook_opt-1.3b"]
columns = ["rf_raw", "rf_chemnlp", "rf_robo", "ft_raw", "ft_chemnlp", "ft_robo"]
# columns = ["rf_raw", "rf_chemnlp", "rf_robo", "ft_raw", "ft_chemnlp"]
# columns = ["rf_raw", "rf_chemnlp", "rf_robo"]
# columns = ["ft_raw", "ft_chemnlp", "ft_robo"]
n_rows = len(rows)
n_cols = len(columns)




# Example data for each subplot
# prop = props[0]

for prop in props:
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(20, 20))
    for i in range(n_rows):
        for j in range(n_cols):
            row = rows[i]
            col = columns[j]
            axes[i, j].set_title(f'{prop}: {row}_{col}', fontsize=12)
            filename = "_".join([col.split("_")[0], row, col.split("_")[1], prop]) + '.csv'
            try:
                df = pd.read_csv(os.path.join("pred", filename))
            except:
                pass
            else:
                preds = df["predictions"].values
                labels = df["labels"].values
                axes[i, j].scatter(labels, preds)
                axes[i, j].plot([-100, 100], [-100, 100], '--r', label='y=x', linewidth=3)
                axes[i, j].set_xlim(min(labels), max(labels))
                axes[i, j].set_ylim(min(labels), max(labels))

# Adjust layout to prevent overlap
plt.tight_layout()

# Display the plot
plt.show()


### Parity plot of predicted and actual property values

In [ ]:

props = ['formation_energy_peratom','ehull','magmom_outcar','spillage','slme', 'Tc_supercon']
columns = ["raw_alignn", "alignn", "alignn_matbert-base-cased_chemnlp", "alignn_matbert-base-cased_robo"]
texts = ["ALIGNN Scratch", "ALIGNN-based TL", "ALIGNN-MatBERT-based TL\n ChemNLP", "ALIGNN-MatBERT-based TL\n Robocrystallographer"]
rows = props
n_rows = len(rows)
n_cols = len(columns)




pred_dir = "/path/to/CrossPropertyTL/prediction/"
old_pred_dir = "/path/to/CrossPropertyTL/prediction/"
colors = [   '#80A1C1', '#A77E58','#EEE3AB','#BA3F1D']

fig, axes = plt.subplots(n_rows, n_cols ,figsize=(6 * n_cols, 6 * n_rows))
mae_values = {}
for i in range(n_rows):
    prop = props[i]
    for j in range(n_cols):
        if n_rows == 1:
            axe = axes[j]
        else:
            axe = axes[i, j]
        row = rows[i]
        col = columns[j]
        # axe.set_title(f'{prop}: {row}_{col}', fontsize=12)
        if col == "raw_alignn":
            filepath = f"/path/to/alignn/{prop}/prediction_results_test_set.csv"
        else:

            filename = "_".join([col, "prop", prop, "pred_otf"]) + '.csv'
            # print(os.path.join(pred_dir, filename))
            filepath = os.path.join(old_pred_dir, filename)
            
        try:
    
            df = pd.read_csv(filepath, index_col=0)
        except:
            print("error reading file")
            pass
        
        if col == "raw_alignn":
            preds = df["prediction"].values
            labels = df["target"].values

        else:
            print(filepath)
            print(df.head())
            preds = df["predictions"].values
            labels = df["labels"].values

        x = np.linspace(-100, 100, 100)
        delta = 2

    

        axe.scatter(labels, preds, color=colors[j], s=60, edgecolors="black", label=col)
        axe.plot([-100, 100], [-100, 100], '--', label='y=x', linewidth=3, color = "#BA3F1D")


        axe.set_ylabel(f"Pred {prop}", fontsize=12, fontstyle='italic')
        axe.set_xlabel(f"DFT {prop}", fontsize=12, fontstyle='italic')
        axe.set_xlim(min(labels), max(labels))
        axe.set_ylim(min(labels), max(labels))
        custom_text = texts[j]
        if i == 0:
            axe.set_title(custom_text, fontsize=11)
        # Calculate MAE values and store them
        mae = np.abs(preds - labels)
        mae_values[col] = mae
    # Calculate the midpoint for annotation
        mid_x = (min(labels) + max(labels)) * 0.35
        mid_y = (min(preds) + max(preds)) * 0.8
# Adjust layout to prevent overlap
plt.tight_layout()
plt.savefig("all_scatter.png", dpi=600, bbox_inches='tight')
# Display the plot
plt.show()
